In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/multilingualabusivecomment/ShareChat-IndoML-Datathon-NSFW-CommentChallenge_Test_20_Percent_NoLabel.csv
/kaggle/input/multilingualabusivecomment/ShareChat-IndoML-Datathon-NSFW-CommentChallenge_Train.csv


In [2]:
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from datasets import load_metric
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForMaskedLM
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.auto import tqdm
import pickle
from sklearn.metrics import accuracy_score, roc_curve, auc
import torch.nn.functional as F

In [3]:
file_location='../input/multilingualabusivecomment/ShareChat-IndoML-Datathon-NSFW-CommentChallenge_Train.csv'
file=open(file_location,'r').read()
total_lines=len(file.split('\n'))
label=[]
text=[]
id=[]
with open(file_location,'r') as file:
    for line in tqdm(file,total=total_lines-1): 
        tokens=line.split(',')
        label.append(tokens[-1])
        id_text=tokens[:2]
        id.append(id_text[0])
        text.append(id_text[1].rstrip('\n'))
df=pd.DataFrame()
labels=[i.strip('\n')[-1] for i in label]
df['id']=id[1:]
df['text']=text[1:]
df['label']=labels[1:]
pd.set_option('display.max_colwidth',-1)
df.head()

    

  0%|          | 0/1500899 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


,id,text,label
0,0,randi ka deewana,1
1,1,👌👌🙏🥰🥰🥰☝️☝️,0
2,2,apni pant utar kar apni chut bhi to bata do madam ji kitni gori hogi,1
3,3,krishnatripathi4578 jesi ghar ki sabhyata vaise sanskar,0
4,4,is randi ko dusra kaam nahi hai kuttiya kahin ki,1


In [4]:
df=df.astype(dtype={"id":int, "text":str,'label':int})
df.info()
df['label'].unique()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500898 entries, 0 to 1500897
Data columns (total 3 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   id      1500898 non-null  int64 
 1   text    1500898 non-null  object
 2   label   1500898 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 34.4+ MB


array([1, 0])

# **Embeddings That Can be Used**
(a). https://www.cfilt.iitb.ac.in/~diptesh/embeddings/README.md
1. as - Assamese
2. bn - Bengali
3. gu - Gujarati
4. hi - Hindi
5. kn - Kannada
6. ko - Konkani
8. ml - Malayalam
9. mr - Marathi
10. ne - Nepali
11. pa - Punjabi
12. sa - Sanskrit
13. ta - Tamil
14. te - Telugu

(b). MuRIL
<ol>
    <li>Assamese</li>
    <li>Bengali</li>
    <li>English</li>
    <li>Gujarati</li>
    <li>Hindi</li>
    <li>Kannada</li>
    <li>Kashmiri</li>
    <li>Malayalam</li>
    <li>Marathi</li>
    <li>Nepali</li>
    <li>Oriya</li>
    <li>Punjabi</li>
    <li>Sanskrit</li>
    <li>Sindhi</li>
    <li>Tamil</li>
    <li>Telugu</li>
    <li>Urdu</li>
    </ol>


In [5]:
def fast_encode(texts, tokenizer, chunk_size=256, maxlen=128):
    
    input_ids = []
    tt_ids = []
    at_ids = []
    
    for i in tqdm(range(0, len(texts), chunk_size)):
        text_chunk = texts[i:i+chunk_size]
        encs = tokenizer(
                    text_chunk,
                    max_length = 128,
                    padding='max_length',
                    truncation=True
                    )
        
        input_ids.extend(encs['input_ids'])
        tt_ids.extend(encs['token_type_ids'])
        at_ids.extend(encs['attention_mask'])
    
    return {'input_ids': input_ids, 'token_type_ids': tt_ids, 'attention_mask':at_ids}

In [6]:
X=df['text']
y=df['label']
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2,random_state=5)
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
train_encoding = fast_encode(list(X_train.values), tokenizer)
val_encoding = fast_encode(list(X_val.values), tokenizer)


Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.02M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/113 [00:00<?, ?B/s]

  0%|          | 0/4691 [00:00<?, ?it/s]

  0%|          | 0/1173 [00:00<?, ?it/s]

In [7]:
class TwitterDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        #item=defaultdict(dict)
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TwitterDataset(train_encoding, list(y_train))
val_dataset = TwitterDataset(val_encoding, list(y_val))
del train_encoding
del val_encoding
del X,y, X_train

In [8]:
import torch.nn as nn
class MurilClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(MurilClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 197285, 128, 2
        #D_in, H, D_out = 1536, 128, 2

        # Instantiate BERT model
        #self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.muril=AutoModelForMaskedLM.from_pretrained("google/muril-base-cased")

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.muril.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.muril(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

In [9]:
from transformers import get_scheduler,AdamW
#from transformers BertForSequenceClassification

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


#model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
#model = AutoModelForMaskedLM.from_pretrained("google/muril-base-cased")
model=MurilClassifier(freeze_bert=False)
model.to(device)
model.train()

batch_size=32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader=DataLoader(val_dataset,batch_size=batch_size)
loss_fn = nn.CrossEntropyLoss()

optim = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_loader)
#progress_bar = tqdm(range(num_training_steps))

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optim,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)



Downloading:   0%|          | 0.00/909M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/muril-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
def train(epoch):
    print("Epoch:",epoch)
    total_loss=0
    with tqdm(train_loader, unit="batch") as tepoch:
        for batch in tepoch:
            outputs=[]
            tepoch.set_description(f"Epoch {epoch}")
            input_ids = batch['input_ids'].to(device)
            #print('input id size',len(input_ids))
            attention_mask = batch['attention_mask'].to(device)
            #print('Attention MAsk Size',len(attention_mask))
            labels = batch['labels'].to(device)
            #print('Label Size',len(labels))
            #outputs = model(input_ids, attention_mask=attention_mask, labels=labels.unsqueeze(1))
            logits=model(input_ids,attention_mask)
            loss=loss_fn(logits,labels)  
            total_loss+=loss.item()
            #loss = outputs[0]
            #print("Training Loss",loss.item())
            loss.backward()
            optim.step()
            lr_scheduler.step()
            optim.zero_grad()
            del input_ids
            del attention_mask
            del labels
            tepoch.set_postfix(loss=loss.item())
    print("Training Loss:",total_loss/batch_size)


In [11]:
best_acc=0
def test(epoch):
    global best_acc
    model.eval()
    true_prediction=0
    total_sample_size=0  
    metric = load_metric("accuracy")
    with tqdm(test_loader, unit="batch") as tepoch:
        for batch in tepoch:
            labels = batch['labels'].to(device)
            input_ids = batch['input_ids'].to(device)            
            attention_mask=batch['attention_mask'].to(device)
            #batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                #out=model(**batch)
                logits=model(input_ids,attention_mask)
            l=loss_fn(logits,labels)
            #l=out.loss
            #prediction=torch.argmax(out.logits,dim=1)
            #metric.add_batch(predictions=prediction, references=batch["labels"])  
            prediction=torch.argmax(logits,dim=1)
            metric.add_batch(predictions=prediction, references=labels)
            tepoch.set_postfix(loss=l.item())
    accuracy=metric.compute()['accuracy']
    print("Test Accuracy:",accuracy)
     # Save checkpoint for the model which yields best accuracy
    if accuracy>=best_acc:
        print("Saving checkpoint with accuracy = ",accuracy)
        best_acc=accuracy
        torch.save({
            'epoch':epoch,
            'model_state_dict':model.state_dict(),
            'optimizer_state_dict':optim.state_dict(),
            'loss':l,
            'accuracy':best_acc
        },'ckpt.pth')

In [12]:
def train_test(epoch):
    best_acc=0
    for i in range(1,epoch+1):
        print("Training")
        train(i)
        print("Testing")
        test(i)
    print("Training ENDS")

In [13]:
train_test(num_epochs)

Training
Epoch: 1


  0%|          | 0/37523 [00:00<?, ?batch/s]

In [ ]:
#model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
import re
def read_csv(path:str):
    file = open(path, "r").read()
    ix = []
    ctx = []

    for row in file.split("\n"):
        l = re.sub(',(?!(?=[^"]*"[^"]*(?:"[^"]*"[^"]*)*$))', "\t", row)
        try:
            lk = l.split("\t")
            if len(lk)>2 and len(lk[0])<6:
                p,q= lk[0], lk[1]
                ix.append(p)
                ctx.append(q)
            elif len(lk)==1 and lk[0]!='':
                #print(lk)
                p=lk[0].split(',')[0].rstrip('\n')
                q=lk[0].split(',')[0].rstrip('\n')
                ix.append(p)
                ctx.append(q)    
            
            else:
                lk=row.replace('"', " ")
                lk=lk.split(",")
                p,q = lk[0], lk[1]
                ix.append(p)
                ctx.append(q)
        except Exception as e:
            print("Exception occurred!.", e)
            print(f"Length of ids obtained: {len(ix)}, and text: {len(ctx)}")

    df = pd.DataFrame()
    df["CommentId"]=ix[1:]
    df["commentText"]=ctx[1:]
    df = df.astype(dtype={"CommentId":int, "commentText":str})
    return df

%time df = read_csv("../input/multilingualabusivecomment/ShareChat-IndoML-Datathon-NSFW-CommentChallenge_Test_20_Percent_NoLabel.csv")
df.head()

In [ ]:
df.tail()

In [ ]:
len(df['CommentId'].unique())

In [ ]:
df.info()

In [ ]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings,length):
        self.encodings = encodings
        self.length=length
        
    def __getitem__(self, idx):
        #item=defaultdict(dict)
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item
    def __len__(self):
        return self.length

In [ ]:
def predict(model,test_loader):
    model.eval()
    all_prediction=[]
    with tqdm(test_loader, unit="batch") as tepoch:
        for batch in tepoch:
            input_ids = batch['input_ids'].to(device)            
            attention_mask=batch['attention_mask'].to(device)
            with torch.no_grad():
                logits=model(input_ids,attention_mask)
            prediction=torch.argmax(logits,dim=1)
            all_prediction.append(prediction)
    return [item.item() for sublist in all_prediction for item in sublist]
          

In [ ]:
X_test=df['commentText']
test_encoding = fast_encode(list(X_test.values), tokenizer)
test_dataset = TestDataset(test_encoding,len(df['CommentId'].unique()))
print(len(test_dataset))
test_loader=DataLoader(test_dataset,batch_size=batch_size)
prediction = predict(model, test_loader)


In [ ]:
submission=pd.DataFrame()
submission['commentId']=df['CommentId']
submission['Label']=prediction

In [ ]:
submission.head()


In [ ]:
submission.to_csv('Submission.csv',index=False,header=True)

In [ ]:
len(submission)